In [ ]:
from owslib.wfs import WebFeatureService
wfs11 = WebFeatureService(url='https://www.imis.bfs.de/ogc/opendata/ows', version='1.1.0')
wfs11.identification.title

In [ ]:
typeName1='opendata:odlinfo_odl_1h_latest'

In [ ]:
# getting dataset1
import json
from owslib.fes import *
from owslib.etree import etree
import xml.dom.minidom 

response1 = wfs11.getfeature(typename=typeName1,bbox=(7.800293,47.709762,13.579102,54.033586), srsname='EPSG:4326', maxfeatures=500, outputFormat='application%2Fjson')
# convert IO-byte to bytes
bytesD=bytes(response1.read())
# convert to json
data1 = json.loads(bytesD)

In [ ]:
#save json to show in qgis
import json
with open(typeName1 + '.json', 'w', encoding='utf-8') as f:
    json.dump(data1, f, ensure_ascii=False, indent=4)

In [ ]:
import pandas as pd
#using pandas for serializing
df1= pd.json_normalize(data1,"features")
df1


In [ ]:
df1["properties.value"] = df1["properties.value"].fillna(value=df1["properties.value"].mean())
df1["properties.height_above_sea"] = df1["properties.height_above_sea"].fillna(value=df1["properties.height_above_sea"].mean())

In [ ]:
#bar plot dataset1

from matplotlib import pyplot as plt

nuclide = df1['properties.name']
value = df1['properties.value']
 
# Figure Size
fig = plt.figure(figsize =(10, 7))
 
# Horizontal Bar Plot
plt.bar(nuclide, value)
 
# Show Plot
plt.show()

In [ ]:
from matplotlib import pyplot as plt
import requests
import numpy as np

x= np.array(df1['properties.value'])
#y=np.array(df1['properties.value_terrestrial'])
y=np.array(df1['properties.height_above_sea'])


plt.scatter(x,y)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats



slope, intercept, r, p, std_err = stats.linregress(x, y)

def myfunc(x):
  return slope * x + intercept

mymodel = list(map(myfunc, x))

plt.scatter(x, y)
plt.plot(x, mymodel)
plt.show()

In [ ]:
import sklearn 
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x.reshape((-1, 1)), y)


In [ ]:
model = LinearRegression().fit(x.reshape((-1, 1)), y)
r_sq = model.score(x.reshape((-1, 1)), y)
print(f"coefficient of determination: {r_sq}")